<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=9c88e96e98d37e91aa17179f52b6f65af1531f5a807a46796530cacdbe7a5be6
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.0
    Uninstalling protobuf-6.33.0:
      Successfully uninstalled protobuf-6.33.0


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-11-07 10:41:35
-------------------
qualified stocks: 87
with latest results: 26
still star stocks: 16
-------------------
Initial Investment:  1.26 C
CY Investment:  1.53 C
Reserve:  1.05 L
Current:  1.42 C
-------------------
Today PnL: -2.66 L (-1.84%)
Current PnL: -22.71 L (-14.88%)
CY Booked + Current PnL: -10.06 L (-6.59%)
-------------------
Total profit:  1.71 L
Total loss:  -24.42 L
-------------------
Total Booked + Current PnL: 16.92 L (13.47%)
Total Booked PnL: 39.63 L (31.55%)
Curr Year Booked PnL: 12.65 L (8.92%)
Prev Year Booked PnL: 26.98 L (21.48%)
Est FTT:  2.32 C
Est FTT PnL: 89.70 L (63.23%)
Deployed:  1.26 C
Current:  1.42 C
CAGR/XIRR %: 7.1%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
58,RELIANCE,1533.00,-6.91,67.0,X-LC,5.81,233267.0,22901.0,5878.0,1.51,10.89,2.52,13.68,37.0,3.90,1.66,29.21,XY25,NTT,REFINERIES
79,UNIONBANK,163.00,-5.43,66.0,M-LC,10.66,171539.0,30699.0,13792.0,-0.44,21.80,8.04,31.59,66.0,2.23,1.22,54.06,XY24,NTT,BANKS
78,TTKPRESTIG,770.00,101.07,51.0,M-SC,1.74,86604.0,-14173.0,14264.0,-1.15,-14.06,16.47,0.09,245.0,-0.99,0.62,13.78,OX40N,NTT,DURABLES
2,ABBOTINDIA,35195.00,-10.86,39.0,X-MC,2.62,88050.0,-2628.0,17531.0,-0.03,-2.90,19.91,16.44,101.0,-0.15,0.63,16.69,X40,ATH,PHARMA
33,ICICIGI,2252.93,-14.54,53.0,X-MC,5.49,146838.0,10705.0,19882.0,-2.73,7.86,13.54,22.47,91.0,0.54,1.04,21.86,X40,ATH,INSURANCE


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
6,ASIANPAINT,3465.66,-9.79,68.0,X-LC,4.62,227744.0,-24024.0,77228.0,4.07,-9.54,33.91,21.13,27.0,-0.31,1.62,22.50,X40,ATH,PAINTS
24,EASEMYTRIP,26.40,-18.31,48.0,M-SC,27.37,78747.0,-98068.0,178866.0,1.77,-55.46,227.14,45.70,197.0,-0.55,0.56,2.80,XY24,NTT,TRAVEL
35,IEX,219.00,-33.63,47.0,H-SC,16.39,189844.0,-10110.0,107338.0,1.63,-5.06,56.54,48.63,115.0,-0.09,1.35,5.91,XR,NTT,MISC
58,RELIANCE,1533.00,-6.91,67.0,X-LC,5.81,233267.0,22901.0,5878.0,1.51,10.89,2.52,13.68,37.0,3.90,1.66,29.21,XY25,NTT,REFINERIES
37,INDIGOPNTS,1408.00,89.73,35.0,M-SC,6.23,124310.0,-50249.0,50283.0,0.73,-28.79,40.45,0.02,221.0,-1.00,0.88,8.13,OX40N,NTT,PAINTS


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
70,SURYODAY,240.0,53.51,42.0,H-SC,15.48,141790.0,-37281.0,103010.0,-8.58,-20.82,72.65,36.71,135.0,-0.36,1.01,40.68,XR,NTT,BANKS
8,ATULAUTO,844.0,3256.76,30.0,M-SC,3.34,150671.0,-46461.0,136282.0,-7.26,-23.57,90.45,45.57,236.0,-0.34,1.07,6.82,XY24,NTT,AUTO
59,REPCOHOME,880.0,-54.55,59.0,H-SC,7.13,262063.0,-24442.0,281770.0,-5.81,-8.53,107.52,89.82,134.0,-0.09,1.86,38.65,XY24,NTT,FINANCE
64,SFL,1287.0,32.15,47.0,M-SC,4.36,174447.0,-87771.0,157595.0,-5.73,-33.47,90.34,26.63,229.0,-0.56,1.24,8.46,XY24,NTT,MISC
19,COFFEEDAY,80.0,-49.70,33.0,L-SC,27.70,73382.0,-40167.0,80214.0,-5.56,-35.37,109.31,35.27,268.0,-0.50,0.52,79.61,XR,NTT,HOTELS


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,TTKPRESTIG,770.0,101.07,51.0,M-SC,1.74,86604.0,-14173.0,14264.0,-1.15,-14.06,16.47,0.09,245.0,-0.99,0.62,13.78,OX40N,NTT,DURABLES
45,KANSAINER,340.0,-67.97,44.0,H-SC,2.67,221346.0,-48321.0,84665.0,-1.05,-17.92,38.25,13.47,138.0,-0.57,1.57,12.55,XY24,NTT,PAINTS
17,CERA,9475.0,-24.34,43.0,H-SC,1.91,138092.0,-37811.0,79831.0,-4.62,-21.50,57.81,23.89,149.0,-0.47,0.98,19.26,OX40N,NTT,CERAMICS
42,JCHAC,2282.0,19921.74,62.0,M-SC,1.52,97843.0,-29962.0,29950.0,-0.02,-23.44,30.61,-0.01,232.0,-1.00,0.69,10.79,OX40N,NTT,AC
67,SIS,528.0,1989.12,38.0,H-SC,2.78,84303.0,-26729.0,50337.0,-0.12,-24.07,59.71,21.26,156.0,-0.53,0.60,13.90,OX40N,NTT,MISC


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
79,UNIONBANK,163.0,-5.43,66.0,M-LC,10.66,171539.0,30699.0,13792.0,-0.44,21.8,8.04,31.59,66.0,2.23,1.22,54.06,XY24,NTT,BANKS


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
61,SAIL,228.00,49.16,57.0,M-MC,9.91,235105.0,10143.0,157520.0,-1.19,4.51,67.00,74.53,192.0,0.06,1.67,38.22,XY24,BTT,STEEL
46,KPIGREEN,731.05,6.33,53.0,H-SC,7.28,126857.0,1560.0,57365.0,-5.38,1.25,45.22,47.03,141.0,0.03,0.90,58.84,MH,ATH,POWER
79,UNIONBANK,163.00,-5.43,66.0,M-LC,10.66,171539.0,30699.0,13792.0,-0.44,21.80,8.04,31.59,66.0,2.23,1.22,54.06,XY24,NTT,BANKS
85,WHIRLPOOL,2270.00,-41.93,48.0,M-SC,5.19,120428.0,5630.0,88406.0,-4.76,4.90,73.41,81.92,223.0,0.06,0.86,42.57,XR,NTT,DURABLES
36,INDIAMART,4810.62,-53.71,52.0,H-SC,5.84,129431.0,6095.0,125535.0,-1.06,4.94,96.99,106.72,119.0,0.05,0.92,29.48,AR,ATH,MISC


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
85,WHIRLPOOL,2270.00,-41.93,48.0,M-SC,5.19,120428.0,5630.0,88406.0,-4.76,4.90,73.41,81.92,223.0,0.06,0.86,42.57,XR,NTT,DURABLES
36,INDIAMART,4810.62,-53.71,52.0,H-SC,5.84,129431.0,6095.0,125535.0,-1.06,4.94,96.99,106.72,119.0,0.05,0.92,29.48,AR,ATH,MISC
46,KPIGREEN,731.05,6.33,53.0,H-SC,7.28,126857.0,1560.0,57365.0,-5.38,1.25,45.22,47.03,141.0,0.03,0.90,58.84,MH,ATH,POWER
31,HINDZINC,730.22,24.68,43.0,M-LC,10.42,202775.0,-2301.0,114872.0,-1.52,-1.12,56.65,54.89,52.0,-0.02,1.44,22.70,X5K,ATH,METALS
86,WIPRO,420.00,-16.52,37.0,M-LC,5.54,146940.0,-4005.0,113467.0,-0.39,-2.65,77.22,72.51,53.0,-0.04,1.04,3.21,XR,NTT,IT


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TATAMOTORS,1065.00,-53.17,21.0,X-LC,28.48,147560.0,-128963.0,239032.0,-2.52,-46.64,161.99,39.81,54.0,-0.54,1.05,4.00,XY24,NTT,AUTO
12,BATAINDIA,2096.00,-41.87,24.0,X-SC,6.96,86951.0,-41719.0,87021.0,-1.74,-32.42,100.08,35.20,219.0,-0.48,0.62,0.00,X40,NTT,FOOTWEAR
23,DMART,5391.45,-20.90,26.0,X-LC,8.25,201550.0,-8857.0,68023.0,-3.62,-4.21,33.75,28.12,38.0,-0.13,1.43,18.42,X40N,ATH,FMCG
44,JSWINFRA,342.00,-23.96,27.0,X-MC,4.68,190991.0,-9356.0,39860.0,-2.33,-4.67,20.87,15.23,178.0,-0.23,1.36,26.07,X40N,NTT,REALTY
30,HINDUNILVR,2922.00,-13.94,30.0,X-LC,6.84,241600.0,-15463.0,50591.0,-1.21,-6.02,20.94,13.67,24.0,-0.31,1.72,13.17,XY25,NTT,FMCG


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
52,PGHH,17907.65,-32.27,31.0,X-MC,0.87,195930.0,-4890.0,72690.0,-2.53,-2.44,37.10,33.76,80.0,-0.07,1.39,2.64,X40,ATH,FMCG
41,ITC,452.00,-38.34,45.0,X-LC,1.13,198901.0,-1237.0,22118.0,-0.53,-0.62,11.12,10.44,4.0,-0.06,1.41,4.91,X40,NTT,FMCG
66,SIEMENS,4671.50,-5.18,39.0,H-LC,1.22,152365.0,-33730.0,81211.0,-2.32,-18.13,53.30,25.51,15.0,-0.42,1.08,12.28,AR,ATH,ELECTRICAL
28,HAVELLS,2069.16,-9.44,33.0,X-MC,3.11,302211.0,-25780.0,132308.0,-3.18,-7.86,43.78,32.48,92.0,-0.19,2.15,2.67,X40,ATH,ELECTRICAL
80,UNITDSPR,1644.00,-6.52,65.0,X-LC,4.12,245100.0,10956.0,37672.0,-1.79,4.68,15.37,20.77,86.0,0.29,1.74,11.92,X40N,NTT,BREWERIES


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
12,BATAINDIA,2096.00,-41.87,24.0,X-SC,6.96,86951.0,-41719.0,87021.0,-1.74,-32.42,100.08,35.20,219.0,-0.48,0.62,0.00,X40,NTT,FOOTWEAR
53,QUESS,986.00,-50.41,33.0,X-SC,37.76,48992.0,-16014.0,166940.0,-2.68,-24.63,340.75,232.18,198.0,-0.10,0.35,0.00,XY24,NTT,MISC
51,PAGEIND,51769.93,-29.85,32.0,X-MC,7.26,79220.0,-4290.0,24321.0,-1.31,-5.14,30.70,23.98,82.0,-0.18,0.56,0.38,X40,ATH,APPARELS
20,COLPAL,3726.84,-7.40,39.0,X-MC,7.33,216380.0,-46985.0,156313.0,-1.23,-17.84,72.24,41.51,84.0,-0.30,1.54,1.54,XY25,ATH,FMCG
3,ACC,3906.00,-38.50,42.0,X-MC,2.45,182400.0,-55551.0,208191.0,-0.47,-23.35,114.14,64.15,174.0,-0.27,1.30,2.32,XY24,BTT,CEMENT


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,QUESS,986.00,-50.41,33.0,X-SC,37.76,48992.0,-16014.0,166940.0,-2.68,-24.63,340.75,232.18,198.0,-0.10,0.35,0.00,XY24,NTT,MISC
51,PAGEIND,51769.93,-29.85,32.0,X-MC,7.26,79220.0,-4290.0,24321.0,-1.31,-5.14,30.70,23.98,82.0,-0.18,0.56,0.38,X40,ATH,APPARELS
57,RELAXO,1176.00,-40.46,41.0,X-SC,4.32,81103.0,-64057.0,139984.0,-0.42,-44.13,172.60,52.31,136.0,-0.46,0.58,8.56,X40N,NTT,FOOTWEAR
12,BATAINDIA,2096.00,-41.87,24.0,X-SC,6.96,86951.0,-41719.0,87021.0,-1.74,-32.42,100.08,35.20,219.0,-0.48,0.62,0.00,X40,NTT,FOOTWEAR
2,ABBOTINDIA,35195.00,-10.86,39.0,X-MC,2.62,88050.0,-2628.0,17531.0,-0.03,-2.90,19.91,16.44,101.0,-0.15,0.63,16.69,X40,ATH,PHARMA


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
75,TCS,4389.96,-29.97,43.0,X-LC,9.90,279622.0,-66334.0,133044.0,-0.52,-19.17,47.58,19.28,1.0,-0.50,1.99,3.37,X40,ATH,IT
39,INFY,2275.00,-20.32,45.0,X-LC,5.14,310362.0,-2526.0,174206.0,-0.74,-0.81,56.13,54.87,3.0,-0.01,2.20,7.41,X40,BTT,IT
41,ITC,452.00,-38.34,45.0,X-LC,1.13,198901.0,-1237.0,22118.0,-0.53,-0.62,11.12,10.44,4.0,-0.06,1.41,4.91,X40,NTT,FMCG
83,VBL,671.64,-16.88,52.0,X-LC,6.90,299158.0,-16684.0,129356.0,-0.45,-5.28,43.24,35.67,5.0,-0.13,2.12,7.84,X40N,ATH,FMCG
1,ABB,7934.00,-42.20,37.0,H-LC,5.25,241488.0,-20131.0,139339.0,-4.06,-7.69,57.70,45.57,7.0,-0.14,1.72,2.76,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
19,COFFEEDAY,80.00,-49.70,33.0,L-SC,27.70,73382.0,-40167.0,80214.0,-5.56,-35.37,109.31,35.27,268.0,-0.50,0.52,79.61,XR,NTT,HOTELS
7,ASIANTILES,137.00,7316.67,62.0,L-SC,10.54,81382.0,-12428.0,89048.0,-2.58,-13.25,109.42,81.67,269.0,-0.14,0.58,57.60,XR,NTT,CERAMICS
49,MASFIN,398.61,-20.05,46.0,H-SC,8.33,90705.0,-7275.0,28880.0,-3.34,-7.42,31.84,22.05,152.0,-0.25,0.64,31.85,XR,ATH,FINANCE
85,WHIRLPOOL,2270.00,-41.93,48.0,M-SC,5.19,120428.0,5630.0,88406.0,-4.76,4.90,73.41,81.92,223.0,0.06,0.86,42.57,XR,NTT,DURABLES
46,KPIGREEN,731.05,6.33,53.0,H-SC,7.28,126857.0,1560.0,57365.0,-5.38,1.25,45.22,47.03,141.0,0.03,0.90,58.84,MH,ATH,POWER


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
81,VAIBHAVGBL,521.00,73.00,63.0,H-SC,2.28,150024.0,-32751.0,143813.0,-3.11,-17.92,95.86,60.77,125.0,-0.23,1.07,34.37,XR,NTT,JEWELLERY
46,KPIGREEN,731.05,6.33,53.0,H-SC,7.28,126857.0,1560.0,57365.0,-5.38,1.25,45.22,47.03,141.0,0.03,0.90,58.84,MH,ATH,POWER
68,SONACOMS,806.63,-32.20,65.0,M-SC,6.92,84376.0,-16783.0,56785.0,-0.34,-16.59,67.30,39.54,202.0,-0.30,0.60,18.98,AR,ATH,AUTO
7,ASIANTILES,137.00,7316.67,62.0,L-SC,10.54,81382.0,-12428.0,89048.0,-2.58,-13.25,109.42,81.67,269.0,-0.14,0.58,57.60,XR,NTT,CERAMICS
62,SAMMAANCAP,326.00,-196.59,67.0,M-SC,33.01,166077.0,16857.0,127331.0,-0.91,11.30,76.67,96.62,208.0,0.13,1.18,79.73,XY25,NTT,FINANCE


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.70
1,25,44.37
2,50,75.50


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    43.72
LC    33.21
MC    23.08
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     29.95
X40      16.25
X40N     12.46
XY25     12.00
XR        9.85
AR        7.81
OX40N     6.53
X200      1.78
X5K       1.44
SR        1.04
MH        0.90
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    24.22
X-LC    22.53
X-MC    18.76
M-SC    12.57
X-SC     5.50
M-LC     5.12
H-LC     4.58
H-MC     2.33
M-MC     1.67
L-SC     1.43
L-LC     0.98
L-MC     0.32
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,15.32,-4.87,38.77
IT,12.45,-19.94,84.19
FINANCE,12.15,-10.51,60.61
MISC,7.76,-18.90,78.87
ELECTRICAL,5.72,-12.25,52.05
PAINTS,5.61,-16.91,34.34
BANKS,4.01,-19.76,89.43
INSURANCE,3.86,-1.64,39.54
AUTO,2.72,-50.24,112.94


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3356333.0,22
XR,1271100.0,13
AR,1187253.0,9
X40,868541.0,12
X40N,714545.0,9
OX40N,565567.0,9
XY25,486221.0,8
SR,274743.0,2
X5K,114872.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3420334.0,24
M-SC,1274132.0,15
X-MC,1233758.0,14
X-LC,1107376.0,13
X-SC,575755.0,6
H-LC,293582.0,3
M-LC,287872.0,4
H-MC,260734.0,2
L-SC,259652.0,3


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1202318.0      6
           AR         842397.0      5
           XR         792907.0      7
M-SC       XY24       736411.0      6
X-MC       XY24       577861.0      4
X-LC       X40        472071.0      5
X-SC       X40N       321794.0      4
X-LC       XY24       316855.0      2
X-MC       X40        309449.0      6
H-SC       SR         274743.0      2
           OX40N      250604.0      3
X-LC       X40N       235051.0      3
H-LC       AR         220550.0      2
X-MC       XY25       188748.0      2
H-MC       XY24       184636.0      1
L-SC       XR         169262.0      2
X-SC       XY24       166940.0      1
X-MC       X40N       157700.0      2
M-MC       XY24       157520.0      1
M-SC       OX40N      148475.0      4
           XR         137609.0      2
           XY25       127331.0      1
           AR         124306.0      2
M-LC       X5K        114872.0      1
           XR         113467.0      1
L-SC       OX40N       90390.0      1
X-SC       X40         87021.0      1
X-LC       XY25        83399.0      3
H-MC       OX40N       76098.0      1
H-LC       X200        73032.0      1
L-MC       XR          57855.0      1
H-SC       MH          57365.0      1
M-LC       XY25        45741.0      1
L-LC       XY25        41002.0      1
M-LC       XY24        13792.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 30.0 seconds
